This notebook is experimenting with two methods of structure optimization: singular value decomposition (SVD) and soft filter pruning (SFP).

The idea of the SVD method is shown in the picture below: the weights of each layer are decomposed into three matrices, then training takes place with additional restrictions on maintaining the orthogonality of the matrices and the sparseness of singular values. After training, the least significant singular numbers and the corresponding rows and columns in orthogonal matrices are pruned. Thus, it is possible to achieve a significant reduction in the number of parameters. During the forward pass, the product of these matrices is calculated, which corresponds to the weight matrix of the original dimension.

![Singular value decomposition](../../docs/img/SVD.png)

The idea of the SVD method is shown in the picture below: after each training epoch, the significance of each filter in the convolutional layer is evaluated, then the least significant filters are set to zero. This procedure is repeated after each training epoch. At the end, unnecessary filters are completely removed from the mozhel.

![Soft filter pruning](../../docs/img/SFP.png)

In [1]:
import sys
sys.path.append('../..')

You can download METU-ALET dataset [here](https://cengmetuedutr-my.sharepoint.com/:u:/g/personal/skalkan_ceng_metu_edu_tr/Ee9IYWHjbWxMrQNuVkuvlU0Buu3DgplFP7BBAWMyG06Qlw?e=z84zLt).
The first step is to prepare the dataset in the pytorch Dataset format.

In [3]:
from torchvision.transforms import ToTensor

from core.operation.utils.object_detection_datasets import COCODataset, collate_fn

dataset_path = '/media/n31v/data/datasets/ALET/'

transform = ToTensor()
train_dataset = COCODataset(images_path=f'{dataset_path}train', json_path=f'{dataset_path}train.json', transform=transform)
val_dataset = COCODataset(images_path=f'{dataset_path}val', json_path=f'{dataset_path}val.json', transform=transform)

100%|██████████| 2191/2191 [00:00<00:00, 438754.84it/s]


In the cell below, we define the parameters of the experiments.

If you want to use a pre-trained model, you can set weights parameter like `weights=<your path to state dict>`.

If you don't want to reproduce experiments, you can skip this cell.

In [4]:
import torch

energy_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99, 0.999, 1]

exp_params = {
    'dataset_name': 'ALET',
    'train_dataset': train_dataset,
    'val_dataset': val_dataset,
    'num_classes': len(train_dataset.classes) + 1,
    'dataloader_params': {'batch_size': 1, 'num_workers': 2, 'collate_fn': collate_fn},
    'model_params': {'weights': 'DEFAULT'},
    'models_saving_path': 'models',
    'optimizer': torch.optim.SGD,
    'optimizer_params': {'lr': 0.005, 'momentum': 0.9, 'weight_decay': 0.0005},
    'scheduler_params': {'step_size': 5, 'gamma': 0.5},
    'target_metric': 'map_50',
    'summary_path': 'runs',
    'summary_per_class': False,
    'gpu': True
}

In the cell below, we run an experiment without structure optimization for 100 epochs.

If you do not want to reproduce experiments, you can skip this cell.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

from core.operation.utils.cv_experimenters import FasterRCNNExperimenter


experimenter = FasterRCNNExperimenter(
    structure_optimization='none',
    structure_optimization_params={},
    **exp_params
)
experimenter.fit(100)

In the cell below, we run an experiment with soft filter pruning optimization for 100 epochs.

If you do not want to reproduce experiments, you can skip this cell.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

from core.operation.utils.cv_experimenters import FasterRCNNExperimenter


experimenter = FasterRCNNExperimenter(
    structure_optimization='SFP',
    structure_optimization_params={
        'pruning_ratio': 0.5,
        'finetuning_epochs': 3
    },
    **exp_params
)
experimenter.fit(100)

Default size: 166.61 MB
ALET/FasterR-CNN/ResNet50_SFP_P-0.50, using device: cuda
Epoch 1


100%|██████████| 262/262 [00:20<00:00, 12.57it/s]


Epoch 2


100%|██████████| 262/262 [00:21<00:00, 12.33it/s]


Model state dict saved.
Epoch 3


...

100%|██████████| 262/262 [00:21<00:00, 12.26it/s]


Model state dict saved.
Epoch 14


100%|██████████| 262/262 [00:20<00:00, 12.57it/s]


Epoch 15


 39%|███▉      | 824/2087 [02:33<03:51,  5.46it/s, loss=0.821]

In the cells below, we run experiments with a singular value decomposition of convolutional layers for 100 epochs.

If you do not want to reproduce experiments, you can skip this cell.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

from core.operation.utils.cv_experimenters import FasterRCNNExperimenter


experimenter = FasterRCNNExperimenter(
    structure_optimization='SVD',
    structure_optimization_params={
        'decomposing_mode': 'spatial',
        'orthogonal_loss_factor': 10,
        'hoer_loss_factor': 0.001,
        'energy_thresholds': energy_thresholds,
        'finetuning_epochs': 3
    },
    **exp_params
)
experimenter.fit(100)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

from core.operation.utils.cv_experimenters import FasterRCNNExperimenter


experimenter = FasterRCNNExperimenter(
    structure_optimization='SVD',
    structure_optimization_params={
        'decomposing_mode': 'channel',
        'orthogonal_loss_factor': 10,
        'hoer_loss_factor': 0.001,
        'energy_thresholds': energy_thresholds,
        'finetuning_epochs': 3
    },
    **exp_params
)
experimenter.fit(100)